# Configuración del entorno

In [0]:
# List
dbutils.fs.mounts()


In [0]:
# Unmount
dbutils.fs.unmount("/mnt/")


/mnt/datalake has been unmounted.


True

In [0]:
# Montar el contenedor de Azure Data Lake Storage (ADLS)
dbutils.fs.mount(
    source = "",
    mount_point = "/mnt/",
    extra_configs = {"": ""}
)



True

In [0]:

# Cargar el dataset desde Azure Data Lake Storage
df = spark.read.csv("/mnt/datalake/retail_dataset.csv", header=True, inferSchema=True)

# Mostrar las primeras filas del dataset
df.show(5)

+---+----------+-----------+--------------+------------+-----+--------+------------+
|_c0|      Date|Customer_ID|Transaction_ID|SKU_Category|  SKU|Quantity|Sales_Amount|
+---+----------+-----------+--------------+------------+-----+--------+------------+
|  1|2016-01-02|       2547|             1|         X52|0EM7L|     1.0|        3.13|
|  2|2016-01-02|        822|             2|         2ML|68BRQ|     1.0|        5.46|
|  3|2016-01-02|       3686|             3|         0H2|CZUZX|     1.0|        6.35|
|  4|2016-01-02|       3719|             4|         0H2|549KK|     1.0|        5.59|
|  5|2016-01-02|       9200|             5|         0H2|K8EHH|     1.0|        6.88|
+---+----------+-----------+--------------+------------+-----+--------+------------+
only showing top 5 rows



# Implementación de particiones avanzadas

Análisis del _dataset_

In [0]:
# Contar el número total de registros
df.count()

# Obtener el esquema del dataset para entender qué columnas utilizar para particionar
df.printSchema()

# Mostrar las primeras filas del dataset para análisis visual
df.show(5)


root
 |-- _c0: integer (nullable = true)
 |-- Date: date (nullable = true)
 |-- Customer_ID: integer (nullable = true)
 |-- Transaction_ID: integer (nullable = true)
 |-- SKU_Category: string (nullable = true)
 |-- SKU: string (nullable = true)
 |-- Quantity: double (nullable = true)
 |-- Sales_Amount: double (nullable = true)

+---+----------+-----------+--------------+------------+-----+--------+------------+
|_c0|      Date|Customer_ID|Transaction_ID|SKU_Category|  SKU|Quantity|Sales_Amount|
+---+----------+-----------+--------------+------------+-----+--------+------------+
|  1|2016-01-02|       2547|             1|         X52|0EM7L|     1.0|        3.13|
|  2|2016-01-02|        822|             2|         2ML|68BRQ|     1.0|        5.46|
|  3|2016-01-02|       3686|             3|         0H2|CZUZX|     1.0|        6.35|
|  4|2016-01-02|       3719|             4|         0H2|549KK|     1.0|        5.59|
|  5|2016-01-02|       9200|             5|         0H2|K8EHH|     1.0|    

# Paso 2: Implementación de particiones avanzadas
### Identificar las columnas clave para particionar:

Basándonos en las columnas, podemos particionar por Date (a nivel de año y mes) y SKU_Category. Estas son columnas que típicamente son consultadas en un análisis de ventas.
### Particionar el dataset:

Particionar los datos por SKU_Category y Date (dividido en año y mes) mejora las consultas que filtran por estos valores. Esto evita leer datos innecesarios:

In [0]:
from pyspark.sql.functions import year, month

df_with_date = df.withColumn("year", year(df["Date"])).withColumn("month", month(df["Date"]))
df_repartitioned = df_with_date.repartition("year", "month", "SKU_Category")


### Guardar el dataset particionado en Delta Lake:

**¿Por qué?:** Delta Lake es un formato de almacenamiento transaccional sobre almacenamiento distribuido como ADLS. Guardar los datos en formato Delta nos da la ventaja de las transacciones ACID, la corrección automática de archivos pequeños y las capacidades de optimización que mejoran el rendimiento de consultas.

In [0]:
df_repartitioned.write.partitionBy("year", "month", "SKU_Category").mode("overwrite").format("delta").save("/mnt/datalake/delta_partitioned_data/")


# Paso 3: Compactación de archivos pequeños con Delta Lake
### Problema: Archivos pequeños:

Cuando se aplican muchas particiones, es común generar archivos pequeños que ralentizan el rendimiento. Esto ocurre porque cada partición puede tener pocos datos si las particiones no están bien distribuidas.
### Solución: Compactar archivos pequeños con OPTIMIZE:

**¿Por qué?:** El comando OPTIMIZE en Delta Lake reduce el número de archivos pequeños, agrupándolos en archivos más grandes. Esto mejora significativamente el rendimiento de las consultas porque Spark necesita leer menos archivos.

In [0]:
spark.sql("OPTIMIZE '/mnt/datalake/delta_partitioned_data/'")


DataFrame[path: string, metrics: struct<numFilesAdded:bigint,numFilesRemoved:bigint,filesAdded:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,filesRemoved:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,partitionsOptimized:bigint,zOrderStats:struct<strategyName:string,inputCubeFiles:struct<num:bigint,size:bigint>,inputOtherFiles:struct<num:bigint,size:bigint>,inputNumCubes:bigint,mergedFiles:struct<num:bigint,size:bigint>,numOutputCubes:bigint,mergedNumCubes:bigint>,clusteringStats:struct<inputZCubeFiles:struct<numFiles:bigint,size:bigint>,inputOtherFiles:struct<numFiles:bigint,size:bigint>,inputNumZCubes:bigint,mergedFiles:struct<numFiles:bigint,size:bigint>,numOutputZCubes:bigint>,numBins:bigint,numBatches:bigint,totalConsideredFiles:bigint,totalFilesSkipped:bigint,preserveInsertionOrder:boolean,numFilesSkippedToReduceWriteAmplification:bigint,numBytesSkippedToReduceWriteAmplification:bigint,startTimeMs:bigint,endTimeMs:bigint,

### Verificar la compactación:

Usa Spark UI para verificar que el número de archivos ha disminuido y que las tareas de lectura son más rápidas tras la compactación.

# Paso 4: Ajuste dinámico de particiones
### Ajuste dinámico del número de particiones:

**¿Por qué?:** Dependiendo del tamaño del clúster y el volumen de datos, ajustar dinámicamente el número de particiones permite que Spark utilice de manera eficiente los recursos del clúster. Si tienes pocas particiones, no aprovecharás completamente el paralelismo. Si tienes demasiadas particiones, estarás sobrecargando el sistema con operaciones de lectura innecesarias.

Para optimizar el rendimiento en clústeres de diferentes tamaños, ajusta dinámicamente el número de particiones. Usa la configuración por defecto de Spark o ajusta manualmente el número de particiones según el tamaño del dataset:

In [0]:
num_partitions = spark.conf.get("spark.sql.shuffle.partitions", "200")

# Reparticionar dinámicamente basándose en el tamaño del clúster
df_dynamic = df.repartition(int(num_partitions))
df_dynamic.write.mode("overwrite").format("delta").save("/mnt/datalake/delta_dynamic_partitioned_data/")

**Objetivo:** Ajustar el número de particiones asegura que el clúster no tenga sobrecarga de particiones, ni pocas particiones que reduzcan el paralelismo.

# Paso 5: Comparar el rendimiento con y sin particiones
### Ejecutar consultas sin particionamiento:

Mide el tiempo de una consulta sin particionamiento para ver la diferencia:

In [0]:
import time
start_time = time.time()
df.groupBy("SKU_Category").agg({"Sales_Amount": "sum"}).show()
print("--- %s seconds ---" % (time.time() - start_time))


+------------+------------------+
|SKU_Category| sum(Sales_Amount)|
+------------+------------------+
|         TEU|1270.0199999999995|
|         6KO|            200.45|
|         RU6| 8358.180000000013|
|         RML| 5764.909999999999|
|         MOE|4340.1300000000065|
|         FU5|25877.709999999934|
|         H1H|            2181.7|
|         WHL| 858.3900000000001|
|         UCR|             59.97|
|         AEB|2171.2400000000043|
|         PBV|10236.329999999998|
|         69B| 2752.180000000002|
|         1EO|          34638.22|
|         9ZX|42629.309999999925|
|         IW1|            414.63|
|         QCG|19855.560000000078|
|         HXR|125.19000000000003|
|         HW0|350.15000000000015|
|         X52|12769.070000000002|
|         01F|38029.870000000075|
+------------+------------------+
only showing top 20 rows

--- 0.7784538269042969 seconds ---


### Ejecutar consultas con particionamiento:

Realiza la misma consulta después de particionar y mide el tiempo:

In [0]:
df_part = spark.read.format("delta").load("/mnt/datalake/delta_partitioned_data/")
start_time = time.time()
df_part.groupBy("SKU_Category").agg({"Sales_Amount": "sum"}).show()
print("--- %s seconds ---" % (time.time() - start_time))


+------------+------------------+
|SKU_Category| sum(Sales_Amount)|
+------------+------------------+
|         FU5| 25877.71000000002|
|         X52|12769.069999999987|
|         01F|38029.870000000024|
|         29A|22307.120000000043|
|         LSD| 16732.67999999999|
|         P42|32009.439999999762|
|         Q4N|31928.339999999986|
|         J4R|28916.230000000043|
|         8HU| 35528.14000000027|
|         1VL|16931.529999999973|
|         XG4| 14422.89000000001|
|         0H2|29875.140000000087|
|         JPI| 4933.950000000004|
|         6BZ| 114061.3300000002|
|         IEV|37618.219999999885|
|         N8U| 49119.54999999983|
|         YMJ|14725.430000000018|
|         OBD|20847.279999999995|
|         U5F| 28230.64999999998|
|         R6E| 34784.58999999996|
+------------+------------------+
only showing top 20 rows

--- 3.395134210586548 seconds ---


### Evaluar el rendimiento:

Compara los tiempos de consulta con y sin particiones. Deberías ver una mejora significativa en los tiempos de respuesta al usar particiones adecuadas.